In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxpath=os.path.sep.join([r'C:\Users\infinity\Desktop\Mask Detection\deploy.prototxt'])
weightspath=os.path.sep.join([r'C:\Users\infinity\Desktop\Mask Detection\res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
prototxpath

'C:\\Users\\infinity\\Desktop\\Mask Detection\\deploy.prototxt'

In [4]:
weightspath

'C:\\Users\\infinity\\Desktop\\Mask Detection\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net= cv2.dnn.readNet(prototxpath, weightspath)

In [6]:
model=load_model(r'C:\Users\infinity\Desktop\Mask Detection\model\maskdetection_model.model')

In [7]:
image= cv2.imread(r'C:\Users\infinity\Desktop\Mask Detection\33.jpg')

In [8]:
image

array([[[191, 219, 213],
        [190, 218, 212],
        [189, 217, 211],
        ...,
        [253, 253, 253],
        [253, 254, 252],
        [253, 254, 252]],

       [[191, 219, 213],
        [191, 219, 213],
        [190, 218, 212],
        ...,
        [253, 253, 253],
        [253, 254, 252],
        [253, 254, 252]],

       [[194, 220, 214],
        [193, 219, 213],
        [192, 218, 212],
        ...,
        [253, 253, 253],
        [253, 254, 252],
        [253, 254, 252]],

       ...,

       [[219, 208, 240],
        [217, 206, 238],
        [214, 203, 235],
        ...,
        [103, 125, 197],
        [ 96, 115, 190],
        [ 93, 109, 185]],

       [[212, 208, 237],
        [214, 211, 237],
        [217, 214, 240],
        ...,
        [113, 135, 207],
        [111, 133, 205],
        [108, 129, 204]],

       [[210, 209, 235],
        [212, 211, 237],
        [215, 215, 239],
        ...,
        [113, 135, 207],
        [109, 133, 205],
        [106, 129, 204]]

In [9]:
image.shape

(194, 259, 3)

In [10]:
(h,w)=image.shape[:2]

In [11]:
blob=cv2.dnn.blobFromImage(image, 1.0, (300,300), (104.0, 177.0, 123.0))

In [12]:
image

array([[[191, 219, 213],
        [190, 218, 212],
        [189, 217, 211],
        ...,
        [253, 253, 253],
        [253, 254, 252],
        [253, 254, 252]],

       [[191, 219, 213],
        [191, 219, 213],
        [190, 218, 212],
        ...,
        [253, 253, 253],
        [253, 254, 252],
        [253, 254, 252]],

       [[194, 220, 214],
        [193, 219, 213],
        [192, 218, 212],
        ...,
        [253, 253, 253],
        [253, 254, 252],
        [253, 254, 252]],

       ...,

       [[219, 208, 240],
        [217, 206, 238],
        [214, 203, 235],
        ...,
        [103, 125, 197],
        [ 96, 115, 190],
        [ 93, 109, 185]],

       [[212, 208, 237],
        [214, 211, 237],
        [217, 214, 240],
        ...,
        [113, 135, 207],
        [111, 133, 205],
        [108, 129, 204]],

       [[210, 209, 235],
        [212, 211, 237],
        [215, 215, 239],
        ...,
        [113, 135, 207],
        [109, 133, 205],
        [106, 129, 204]]

In [13]:
blob.shape

(1, 3, 300, 300)

In [14]:
net.setInput(blob)
detection= net.forward()

In [15]:
detection

array([[[[0.        , 1.        , 0.99060166, ..., 0.38913184,
          0.9756048 , 1.2486811 ],
         [0.        , 1.        , 0.81932384, ..., 0.07317504,
          0.652816  , 0.90807915],
         [0.        , 1.        , 0.2483609 , ..., 2.045261  ,
          2.6716063 , 2.9189897 ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [16]:
#loop over the detections
for i in range(0,detection.shape[2]):
    confidence=detection[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detection[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()